In [1]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

ml_client = MLClient.from_config(DefaultAzureCredential())
ml_client


Found the config file in: /config.json
Class DeploymentTemplateOperations: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x75a4e4972770>,
         subscription_id=32630fbe-e626-46be-be39-56432474f4be,
         resource_group_name=ruap-cars-v01,
         workspace_name=ruap-cars-pred)

In [3]:
from azure.ai.ml.entities import Model

model_asset = Model(
    path="model_package",
    name="used-car-model-package",
    description="RandomForest model + preprocessor (OHE+Scaler) for used car price prediction",
    type="custom_model"
)

model_asset = ml_client.models.create_or_update(model_asset)
model_asset.name, model_asset.version


Uploading model_package (57.38 MBs): 100%|██████████| 57383443/57383443 [00:01<00:00, 47614899.69it/s]




('used-car-model-package', '1')

In [4]:
import time
from azure.ai.ml.entities import ManagedOnlineEndpoint, ManagedOnlineDeployment, Environment, CodeConfiguration
from azure.ai.ml.constants import AssetTypes

endpoint_name = "usedcar-endpoint-" + str(int(time.time()))
endpoint = ManagedOnlineEndpoint(
    name=endpoint_name,
    description="Used car price prediction endpoint"
)

ml_client.online_endpoints.begin_create_or_update(endpoint).result()
endpoint_name


/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/mlflow/__init__.py:41: UserWarning: Versions of mlflow (3.8.1) and child packages mlflow-skinny (3.5.0) are different. This may lead to unexpected behavior. Please install the same version of all MLflow packages.
  mlflow.mismatch._check_version_mismatch()


'usedcar-endpoint-1770148704'

In [7]:
# obriši deployment (ako postoji)
ml_client.online_deployments.begin_delete(
    name="blue",
    endpoint_name=endpoint_name
).result()


In [8]:
from azure.ai.ml.entities import Environment

env = Environment(
    name="used-car-env",
    description="Conda env for used car scoring",
    conda_file="azure_endpoint/conda.yaml",
    image="mcr.microsoft.com/azureml/minimal-ubuntu20.04-py38-cpu-inference:latest"
)


In [2]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

ml_client = MLClient.from_config(DefaultAzureCredential())
ml_client


Found the config file in: /config.json
Class DeploymentTemplateOperations: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x74bf19657190>,
         subscription_id=32630fbe-e626-46be-be39-56432474f4be,
         resource_group_name=ruap-cars-v01,
         workspace_name=ruap-cars-pred)

In [4]:
endpoint_name = "usedcar-endpoint-1770148704"


In [5]:
print(
    ml_client.online_deployments.get_logs(
        name="blue",
        endpoint_name=endpoint_name,
        lines=2000
    )
)


Instance status:
SystemSetup: Succeeded
UserContainerImagePull: Succeeded
ModelDownload: Succeeded
UserContainerStart: InProgress

Container events:
Kind: Pod, Name: Downloading, Type: Normal, Time: 2026-02-03T20:21:23.680873Z, Message: Start downloading models
Kind: Pod, Name: Pulling, Type: Normal, Time: 2026-02-03T20:21:24.629209Z, Message: Start pulling container image
Kind: Pod, Name: Pulled, Type: Normal, Time: 2026-02-03T20:22:10.863171Z, Message: Container image is pulled successfully
Kind: Pod, Name: Downloaded, Type: Normal, Time: 2026-02-03T20:22:10.863171Z, Message: Models are downloaded successfully
Kind: Pod, Name: Created, Type: Normal, Time: 2026-02-03T20:22:10.909427Z, Message: Created container inference-server
Kind: Pod, Name: Started, Type: Normal, Time: 2026-02-03T20:22:10.95572Z, Message: Started container inference-server
Kind: Pod, Name: LivenessProbeFailed, Type: Warning, Time: 2026-02-03T20:22:20.341775Z, Message: Liveness probe failed: HTTP probe failed with 

In [7]:
model_asset = ml_client.models.get(name="used-car-model-package", label="latest")
model_asset


Model({'job_name': None, 'intellectual_property': None, 'system_metadata': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'used-car-model-package', 'description': 'RandomForest model + preprocessor (OHE+Scaler) for used car price prediction', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/32630fbe-e626-46be-be39-56432474f4be/resourceGroups/ruap-cars-v01/providers/Microsoft.MachineLearningServices/workspaces/ruap-cars-pred/models/used-car-model-package/versions/1', 'Resource__source_path': '', 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/dpericcars/code/Users/dperic1/used_car_project', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x74bf0b3ce560>, 'serialize': <msrest.serialization.Serializer object at 0x74bf0b576c50>, 'version': '1', 'latest_version': None, 'path': 'azureml://subscriptions/32630fbe-e626-46be-be39-56432474f4be/resourceGroups/ruap-cars-v01/works

In [10]:
from azure.ai.ml.entities import Environment

aml_env = Environment(
    name="used-car-env",
    description="Conda env for used car scoring",
    conda_file="azure_endpoint/conda.yaml",
    image="mcr.microsoft.com/azureml/minimal-ubuntu20.04-py38-cpu-inference:latest"
)


In [11]:
from azure.ai.ml.entities import ManagedOnlineDeployment, CodeConfiguration

deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=endpoint_name,
    model=model_asset,
    environment=aml_env,
    code_configuration=CodeConfiguration(code="azure_endpoint", scoring_script="score.py"),
    instance_type="Standard_DS1_v2",
    instance_count=1,
)

ml_client.online_deployments.begin_create_or_update(deployment).result()


Instance type Standard_DS1_v2 may be too small for compute resources. Minimum recommended compute SKU is Standard_DS3_v2 for general purpose endpoints. Learn more about SKUs here: https://learn.microsoft.com/azure/machine-learning/referencemanaged-online-endpoints-vm-sku-list
Check: endpoint usedcar-endpoint-1770148704 exists
Uploading azure_endpoint (0.0 MBs): 100%|██████████| 2161/2161 [00:00<00:00, 87109.24it/s]


/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/mlflow/__init__.py:41: UserWarning: Versions of mlflow (3.8.1) and child packages mlflow-skinny (3.5.0) are different. This may lead to unexpected behavior. Please install the same version of all MLflow packages.
  mlflow.mismatch._check_version_mismatch()


...........................................

ManagedOnlineDeployment({'private_network_connection': None, 'package_model': False, 'provisioning_state': 'Succeeded', 'endpoint_name': 'usedcar-endpoint-1770148704', 'type': 'Managed', 'name': 'blue', 'description': None, 'tags': {}, 'properties': {'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/32630fbe-e626-46be-be39-56432474f4be/providers/Microsoft.MachineLearningServices/locations/polandcentral/mfeOperationsStatus/odidp:e3db4f3a-3dd0-4926-bc5f-620fa28cfc0a:99bf16ee-e030-4e02-a4ae-ebf19a958de8?api-version=2023-04-01-preview'}, 'print_as_yaml': False, 'id': '/subscriptions/32630fbe-e626-46be-be39-56432474f4be/resourceGroups/ruap-cars-v01/providers/Microsoft.MachineLearningServices/workspaces/ruap-cars-pred/onlineEndpoints/usedcar-endpoint-1770148704/deployments/blue', 'Resource__source_path': '', 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/dpericcars/code/Users/dperic1/used_car_project', 'creation_context': <azure.ai.ml._restclient.v2023_04_0

In [12]:
endpoint = ml_client.online_endpoints.get(endpoint_name)
endpoint.traffic = {"blue": 100}
ml_client.online_endpoints.begin_create_or_update(endpoint).result()


Readonly attribute principal_id will be ignored in class <class 'azure.ai.ml._restclient.v2022_05_01.models._models_py3.ManagedServiceIdentity'>
Readonly attribute tenant_id will be ignored in class <class 'azure.ai.ml._restclient.v2022_05_01.models._models_py3.ManagedServiceIdentity'>


ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://usedcar-endpoint-1770148704.polandcentral.inference.ml.azure.com/score', 'openapi_uri': 'https://usedcar-endpoint-1770148704.polandcentral.inference.ml.azure.com/swagger.json', 'name': 'usedcar-endpoint-1770148704', 'description': 'Used car price prediction endpoint', 'tags': {}, 'properties': {'createdBy': 'Dominik Perić', 'createdAt': '2026-02-03T19:58:45.924423+0000', 'lastModifiedAt': '2026-02-03T22:27:12.377667+0000', 'azureml.onlineendpointid': '/subscriptions/32630fbe-e626-46be-be39-56432474f4be/resourcegroups/ruap-cars-v01/providers/microsoft.machinelearningservices/workspaces/ruap-cars-pred/onlineendpoints/usedcar-endpoint-1770148704', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/32630fbe-e626-46be-be39-56432474f4be/providers/Microsoft.MachineLearningServices/locations/polandcentral/mfeOperationsStatus/oeidp:e3db4f3a-3dd0-4926-bc5f-620f

In [13]:
endpoint = ml_client.online_endpoints.get(endpoint_name)
keys = ml_client.online_endpoints.get_keys(endpoint_name)

scoring_uri = endpoint.scoring_uri
api_key = keys.primary_key

scoring_uri


'https://usedcar-endpoint-1770148704.polandcentral.inference.ml.azure.com/score'

In [14]:
import requests, json

sample = {
    "brand": "Ford",
    "model": "Utility Police Interceptor Base",
    "model_year": 2013,
    "milage": 51000,
    "fuel_type": "E85 Flex Fuel",
    "engine": "300.0HP 3.7L V6 Cylinder Engine Flex Fuel Capability",
    "transmission": "6-Speed A/T",
    "ext_col": "Black",
    "int_col": "Black",
    "accident": "At least 1 accident or damage reported",
    "clean_title": "Yes"
}

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}

r = requests.post(scoring_uri, headers=headers, data=json.dumps(sample))
print(r.status_code)
print(r.text)


200
{"predicted_price": 15941.5}
